In [39]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.layers import (
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    Input,
    Lambda,
)
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
data_dir = "./data/train/"
labels_path = "./data/labels.csv"
img_size = (224, 224, 3)

In [4]:
df = pd.read_csv(labels_path)
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
df["image_path"] = data_dir + df["id"] + ".jpg"
df = df.drop(columns="id")

In [6]:
breeds = sorted(df["breed"].unique().tolist())
len(breeds)

120

In [7]:
class_to_num = dict(zip(breeds, range(len(breeds))))

<h1> Using Machine Learning

In [27]:
ml_labels = df["breed"]
ml_data_list = []

In [28]:
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    img_pixels = load_img(row["image_path"], target_size=(32, 32, 3))
    img_pixels = img_to_array(img_pixels)
    img_pixels = img_pixels.ravel().reshape(1, -1)

    row_data = pd.DataFrame(
        img_pixels, columns=[f"pixel{i}" for i in range(len(img_pixels[0]))]
    )
    ml_data_list.append(row_data)

100%|██████████| 10222/10222 [00:21<00:00, 476.85it/s]


In [29]:
ml_data = pd.concat(ml_data_list, ignore_index=True)

In [30]:
ml_train_data, ml_test_data, ml_train_label, ml_test_label = train_test_split(
    ml_data.to_numpy(), ml_labels.to_numpy(), test_size=0.2
)

In [33]:
ml_test_data.shape

(2045, 3072)

In [34]:
ml_train_data.shape

(8177, 3072)

In [21]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(ml_train_data, ml_train_label)

KNeighborsClassifier(n_neighbors=1)

In [41]:
lg = LogisticRegression()
lg.fit(ml_train_data, ml_train_label)

In [35]:
y_pred_lg = lg.predict(ml_test_data)

In [37]:
def evaluation_performance(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    pre = precision_score(
        y_true, y_pred, pos_label="positive", average="micro"
    )
    rec = recall_score(y_true, y_pred, pos_label="positive", average="micro")
    return {"Accuracy: ": acc, "Precision: ": pre, "Recall: ": rec}

In [38]:
# evaluation_performance(ml_test_label, y_pred_knn)
evaluation_performance(ml_test_label, y_pred_lg)

C:\Users\ADMIN\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1521: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\ADMIN\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1521: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


{'Accuracy: ': 0.025427872860635695,
 'Precision: ': 0.025427872860635695,
 'Recall: ': 0.025427872860635695}

<h1>Using CNN

Put all images into 1 array n value [n, x1, x2, x3]: <br>
n: number of images<br>
x1: height of image<br>
x2: width of image<br>
x3: depth of image(1 for grayscale image, 3 for color image)


In [ ]:
def image_processing(data: pd.DataFrame, image_size: tuple):
    X = np.zeros(
        [len(data), image_size[0], image_size[1], image_size[2]],
        dtype=np.uint8,
    )
    y = np.zeros([len(data), 1], dtype=np.uint8)
    for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
        img_pixels = load_img(row["image_path"], target_size=image_size)
        X[idx] = img_pixels
        y[idx] = class_to_num[row["breed"]]
    return X, y

In [ ]:
X, y = image_processing(df, img_size)

100%|██████████| 10222/10222 [00:09<00:00, 1087.61it/s]


In [ ]:
X.shape

(10222, 224, 224, 3)

Extract feature using MobileNetV2

In [ ]:
def extract_feature(image_size: tuple, data):
    input_layer = Input(image_size)
    processor = Lambda(preprocess_input)(input_layer)
    base_model = MobileNetV2(
        weights="imagenet", include_top=False, input_shape=image_size
    )(processor)
    maxPool = MaxPooling2D()(base_model)
    feature_extractor = Model(inputs=input_layer, outputs=maxPool)
    feature_maps = feature_extractor.predict(data, batch_size=64, verbose=1)
    return feature_maps

In [ ]:
features = extract_feature(img_size, X)

160/160 [==============================] - 77s 477ms/step


In [ ]:
features.shape

(10222, 3, 3, 1280)

In [ ]:
le = LabelBinarizer()
y = le.fit_transform(y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    features, y, test_size=0.2, random_state=12
)

In [ ]:
X_train = X_train / 255.0
X_val = X_val / 255.0

In [ ]:
X_train.shape[1:]

(3, 3, 1280)

Using Neural Network

In [ ]:
model = Sequential()
model.add(Input(X_train.shape[1:]))

model.add(Flatten())

model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))

model.add(Dense(len(breeds), activation="softmax"))
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 11520)             0         
                                                                 
 dense (Dense)               (None, 256)               2949376   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 120)               15480     
                                                                 
Total params: 2,997,752
Trainable params: 2,997,752
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 20
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=64,
)

Epoch 1/20


128/128 [==============================] - 2s 10ms/step - loss: 3.6853 - accuracy: 0.2266 - val_loss: 1.9662 - val_accuracy: 0.5081
Epoch 2/20
128/128 [==============================] - 1s 9ms/step - loss: 1.4674 - accuracy: 0.5990 - val_loss: 1.1660 - val_accuracy: 0.6699
Epoch 3/20
128/128 [==============================] - 1s 9ms/step - loss: 1.0159 - accuracy: 0.7043 - val_loss: 1.0445 - val_accuracy: 0.6822
Epoch 4/20
128/128 [==============================] - 1s 9ms/step - loss: 0.8151 - accuracy: 0.7515 - val_loss: 0.9507 - val_accuracy: 0.7125
Epoch 5/20
128/128 [==============================] - 1s 9ms/step - loss: 0.6685 - accuracy: 0.7966 - val_loss: 0.9011 - val_accuracy: 0.7213
Epoch 6/20
128/128 [==============================] - 1s 9ms/step - loss: 0.5513 - accuracy: 0.8327 - val_loss: 0.8796 - val_accuracy: 0.7350
Epoch 7/20
128/128 [==============================] - 1s 9ms/step - loss: 0.4578 - accuracy: 0.8603 - val_loss: 0.8348 - val_accuracy: 0.7540
Epoch 8/20
128/1

In [ ]:
test_path = "./data/test/"
img_test_files = os.listdir(test_path)[:5]

In [ ]:
for img in img_test_files:
    file_name = os.path.join(test_path, img)
    image_test = load_img(file_name, target_size=img_size)
    image_test = np.array(image_test)
    image_test = image_test.reshape((1, 224, 224, 3))
    image_test = extract_feature(img_size, image_test)
    breed_index = model.predict(image_test).argmax(axis=-1)
    breed_label = [
        label for label, index in class_to_num.items() if index == breed_index
    ][0]

    print(breed_label)

1/1 [==============================] - 0s 37ms/step
japanese_spaniel
1/1 [==============================] - 0s 16ms/step
samoyed
1/1 [==============================] - 0s 16ms/step
english_setter
1/1 [==============================] - 0s 16ms/step
newfoundland
1/1 [==============================] - 0s 16ms/step
tibetan_terrier
